# Combining my .pkl file with my positives (vira beg) and my .pkl file with my negatives (hand-picked list of neg species)
kernel opso12 (Python 3.11.14) \
positives: /home/brg226/projects/blra/training_data/onehot_pos_virabeg.pkl \
negatives: /home/brg226/projects/blra/training_data/negative_multihot_labels.pkl 

In [ ]:
#OPSO & general imports 
from opensoundscape import Audio, Spectrogram
from opensoundscape.annotations import BoxedAnnotations

# General-purpose packages
import numpy as np
import pandas as pd
from glob import glob
from pathlib import Path

In [ ]:
# First I should check on the column names of the two files and see if they are compatible 
positives = pd.read_pickle('/home/brg226/projects/blra/training_data/onehot_pos_virabeg.pkl')
negatives = pd.read_pickle('/home/brg226/projects/blra/training_data/negative_multihot_labels.pkl')

In [ ]:
# make the file, start_time, end_time columns columns and not indexes for both dataframes
positives = positives.reset_index()
negatives=negatives.reset_index()
# I will have to set_index and specify the columns in a list that you want to be the index at the end of this notebook

In [ ]:
print(positives.columns)
print(negatives.columns)

Okay looks like the file and start_time columns should be file, I need to add an end_time column to the natives file which will just be start_time + 1.5 s and I need to figure out if we have a problem since the posiitves aee designated with "true" and in the negatives file they are designated with a 1. \
I also need to make it so that negatives get filled in with 0s when I concatenate the two dataframes, i.e. the virail column will be new to the negatives dataframe and there will need to be 0s until it is the positive clips.

In [ ]:
# adding end_time column to negatives dataframe by adding 1.5 to start_time
negatives['end_time'] = negatives['start_time'] + 1.5
print(negatives.columns)

In [ ]:
#changing values of "True" in virail column to 1
positives['virail'] = positives['virail'].map({True:1, 'true':1, 'True':1, 'TRUE':1, 1:1}).fillna(0).astype(int)

In [ ]:
positives.head(10)

In [ ]:
# delete the viral column that I accidentally created lol
if 'viral' in positives.columns:
    del positives['viral']
positives.head(10)

In [ ]:
# examine first 5 rows of negatives and positives df again and see if we are ready to merge
print(positives.head())
print(negatives.head())


In [ ]:
# looks good, we need to make the file, start_time, end_time columns the index for both dataframes now
positives = positives.set_index(['file', 'start_time', 'end_time'])
negatives = negatives.set_index(['file', 'start_time', 'end_time'])


In [ ]:
# print index values for both dataframes to confirm
print(positives.index)
print(negatives.index)

In [ ]:
# I need to merge the dataframes into a new .pkl file called training_data_combined.pkl, where virail=1 I need the rest of the row to =0
combined = pd.concat([positives, negatives], axis=0, ignore_index=False).fillna(0)
# verify the combined dataframe
print(combined.head(10))


In [ ]:
print(combined.tail(10))


In [ ]:
print(combined.index)
print(combined.columns)
print(combined.shape)

In [ ]:
# save the combined dataframe as a .pkl file
out = "/home/brg226/projects/blra/training_data/training_data_combined.pkl"
combined.to_pickle(out)
print("Saved:", out)

In [ ]:
combo_saved = pd.read_pickle("/home/brg226/projects/blra/training_data/training_data_combined.pkl")
print(combo_saved.head())
print(combo_saved.index)
print(combo_saved.columns)
print(combo_saved.shape)
print(combo_saved.tail())